In [1]:
immutable Transformation
    self :: Function
    deriv :: Function
end

function Base.show(io :: IO, m :: Transformation)
    print(io, string("Transformation(", m.self, ")"))
end

import Core.eval

function eval(transformation :: Transformation)
    transformation.self
end

function deriv(transformation :: Transformation)
    transformation.deriv
end

function sigmoid(x)
        1. / (1. + exp(-x)) - 0.5
end

function dsigmoid(x)
    exp(-x) / (1. + exp(-x)) ^ 2
end

const Sigmoid = Transformation(sigmoid, dsigmoid)

const Identity = Transformation(identity, one) 

Transformation(identity)

In [2]:
function calc_norm(array :: Matrix{Float64})
  sqrt(sumabs2(array) / size(array, 1))
end

function calc_norm(vector :: Array{Float64, 1})
  sqrt(sumabs2(vector))
end

function restriction(vector)
  vector / (calc_norm(vector) + 0.001)
end

function init(a...)
    restriction(randn(a...))
end

const DefaultRate = 0.01

immutable Class
    class_name :: String
    type_len :: Int64
end

function Base.show(io :: IO, m :: Class)
    print(io, string("Class(", m.class_name, ")"))
end

immutable Object
    obj_name :: String
    class :: Class
    value :: Array{Float64, 1}
end

Object(obj_name :: String, class :: Class) = Object(obj_name, class, init(class.type_len))

function Base.show(io :: IO, m :: Object)
    print(io, string("Obj(", m.obj_name, "){", m.class.class_name, "}"))
end

type DFunction
    func_name :: String
    in_classes :: Array{Class, 1}
    out_class :: Class
    f_matrix :: Matrix{Float64}
    f_transformation :: Transformation
    learning_rate :: Float64
end

function Base.show(io :: IO, m :: DFunction)
    in_names = join([c.class_name for c in m.in_classes], ",")
    print(io, string("Func(", m.func_name, "){", in_names, "->", m.out_class.class_name, "}"))
end

function LFunction(func_name, in_classes :: Array{Class, 1}, out_class :: Class, learning_rate = DefaultRate)
    in_len = sum([class.type_len for class in in_classes])
    out_len = out_class.type_len
    DFunction(func_name, in_classes, out_class, init(out_len, in_len), Identity, learning_rate)
end

function SFunction(func_name, in_classes :: Array{Class, 1}, out_class :: Class, learning_rate = DefaultRate)
    in_len = sum([class.type_len for class in in_classes])
    out_len = out_class.type_len
    DFunction(func_name, in_classes, out_class, init(out_len, in_len), Sigmoid, learning_rate)
end

function DFunction(func_name, in_classes :: Array{Class, 1}, out_class :: Class, f_transformation = Sigmoid, learning_rate = DefaultRate)
    in_len = sum([class.type_len for class in in_classes])
    out_len = out_class.type_len
    DFunction(func_name, in_classes, out_class, init(out_len, in_len), f_transformation, learning_rate)
end

function CoFunction(func_name, in_class :: Class, f_transformation = Sigmoid)
    DFunction(func_name, [in_class], in_class, diagm(ones(in_class.type_len)), f_transformation, 0.0)
end

function apply(func :: DFunction, objs :: Array{Object, 1})
    inputs = vcat([obj.value for obj in objs]...)
    outputs = eval(func.f_transformation).(func.f_matrix * inputs)
    Object("untitled", func.out_class, outputs)
end

function apply!(func :: DFunction, in_objs :: Array{Object, 1}, out_obj :: Object)
    inputs = vcat([obj.value for obj in in_objs]...)
    out_obj.value .= eval(func.f_transformation).(func.f_matrix * inputs)
end

apply! (generic function with 1 method)

In [3]:
Sensor = Class("Sensor", 100)
sensor = Object("sensor", Sensor)
Action = Class("Action", 120)
action = Object("action", Action)
act = DFunction("act", [Sensor, Sensor, Action], Action)
apply(act, [sensor, sensor, action])

Obj(untitled){Action}

In [4]:
type Tree
    op
    value
    subtrees :: Array{Tree, 1}
end

function Base.show(io :: IO, m :: Tree)
    if length(m.subtrees) > 0
        print(io, "[")
    end
    print(io, string(m.op))
    if length(m.subtrees) > 0
        print(io, ", ")
    end
    subs = join([string(subm) for subm in m.subtrees], ", ")
    print(io, subs)
    if length(m.subtrees) > 0
        print(io, "]")
    end
end

function beautify(m :: Tree, indent :: Int64)
    indentation = join(["    " for i in 1:indent])
    string(indentation, m.op, "\n", join([beautify(sub, indent + 1) for sub in m.subtrees]))
end

beautify(m :: Tree) = beautify(m, 0)

pprint(m) = print(beautify(m))
    
function _bottom_up(func :: Function, tree :: Tree, dict :: Dict)
    function f()
        func(tree, [_bottom_up(func, t, dict) for t in tree.subtrees])
    end
    get!(f, dict, tree)
end

## bottom_up(func :: Function, tree :: Tree) = _bottom_up(func, tree, Dict())

function bottom_up(func :: Function, tree :: Tree)
    func(tree, [bottom_up(func, t) for t in tree.subtrees])
end

bottom_up (generic function with 1 method)

In [5]:
function lens_split(x :: Array, lens :: Array{Int64, 1})
    ind = cumsum(lens)
    n = length(ind)
    ind = [0; ind]
    [view(x, (ind[i]+1):ind[i+1]) for i in 1:n]
end

function init_node!(op :: Object, value :: Dict)
    value[:value] = Array{Float64, 1}(op.class.type_len)
    value[:d] = Array{Float64, 1}(op.class.type_len)
end

function init_node!(op :: DFunction, value :: Dict)
    value[:pre_matrix] = Array{Float64, 1}(size(op.f_matrix, 2))
    value[:pre_transform] = Array{Float64, 1}(op.out_class.type_len)
    value[:value] = Array{Float64, 1}(op.out_class.type_len)
    value[:inputs] = lens_split(value[:pre_matrix], [c.type_len for c in op.in_classes])
    value[:d] = Array{Float64, 1}(op.out_class.type_len)
    value[:pre_transform_d] = Array{Float64, 1}(op.out_class.type_len)
    value[:pre_matrix_d] = Array{Float64, 1}(size(op.f_matrix, 2))
    value[:input_ds] = lens_split(value[:pre_matrix_d], [c.type_len for c in op.in_classes])
end

function _init_tree!(tree :: Tree, _)
    init_node!(tree.op, tree.value)
    tree
end
    
function init_tree!(tree :: Tree)
    bottom_up(_init_tree!, tree)
end

function eval_node!(op :: Object, value :: Dict, value_list)
    value[:value] .= op.value
end

function eval_node!(op :: DFunction, value :: Dict, value_list)
    i = 1
    for v in value_list
        value[:inputs][i] .= v[:value]
        i += 1
    end
    ## value[:pre_matrix] .= vcat([v[:post_feature] for v in value_list]...)
    A_mul_B!(value[:pre_transform], op.f_matrix, value[:pre_matrix])
    value[:value] .= eval(op.f_transformation).(value[:pre_transform])
end

function _eval_tree!(tree :: Tree, _)
    eval_node!(tree.op, tree.value, (t.value for t in tree.subtrees))
    tree
end
    
## function eval_tree!(tree :: Tree)
##     bottom_up(_eval_tree!, tree)
## end
    
function eval_tree!(tree :: Tree)
    foreach(eval_tree!, tree.subtrees)
    eval_node!(tree.op, tree.value, (t.value for t in tree.subtrees))
    tree
end

eval_tree! (generic function with 1 method)

In [6]:
Sensor = Class("Sensor", 100)
sensor = Object("sensor", Sensor)
Action = Class("Action", 120)
action = Object("action", Action)
act = DFunction("act", [Sensor, Action], Action)
tree = Tree(act, Dict(), [Tree(sensor, Dict(), []), Tree(action, Dict(), [])])
init_tree!(tree)
sum(abs(eval_tree!(tree).value[:value] - apply(act, [sensor, action]).value))

0.0

In [7]:
function bp_transformation!(transformation, inputs, d, new_d)
    new_d .= deriv(transformation).(inputs)
    for i in 1:length(d)
        new_d[i] *= d[i]
    end
end

function bp_matrix!(inputs, matrix, d, step, new_d)
    ## step = 0.01
    ## dmatrix = reshape(d, (length(d), 1)) * reshape(inputs, (1, length(inputs)))
    ## matrix[:, :] += step * reshape(d, (length(d), 1)) * reshape(inputs, (1, length(inputs)))
    for j in 1:size(matrix, 2)
        new_d[j] = 0.0
        for i in 1:size(matrix, 1)
            matrix[i, j] += step * d[i] * inputs[j]
            new_d[j] += matrix[i, j] * d[i]
        end
    end
end

function bp_function!(op :: DFunction, value :: Dict)
    bp_transformation!(op.f_transformation, value[:pre_transform], value[:d], value[:pre_transform_d])
    bp_matrix!(value[:pre_matrix], op.f_matrix, value[:pre_transform_d], op.learning_rate, value[:pre_matrix_d])
end

function bp_tree!(tree :: Tree)
    if typeof(tree.op) == DFunction
        bp_function!(tree.op, tree.value)
        for i in 1:length(tree.subtrees)
            tree.subtrees[i].value[:d] .= tree.value[:input_ds][i]
            bp_tree!(tree.subtrees[i])
        end
        ## ds = lens_split(d, [c.feature_len for c in tree.op.in_classes])
        ## for i in 1:length(tree.subtrees)
        ##     bp_tree!(tree.subtrees[i], tree.value[:ds][i])
        ## end
        ## dds = map(bp_class!, tree.op.in_classes, [t.value for t in tree.subtrees], tree.value[:ds])
        ## foreach(bp_tree!, tree.subtrees, dds)
    end
end

bp_tree! (generic function with 1 method)

In [8]:
Sensor = Class("Sensor", 100)
sensor = Object("sensor", Sensor)
Action = Class("Action", 120)
action = Object("action", Action)
act = DFunction("act", [Sensor, Action], Action)
tree = Tree(act, Dict(), [Tree(sensor, Dict(), []), Tree(action, Dict(), [])])
init_tree!(tree)
d = ones(length(eval_tree!(tree).value[:value])) - eval_tree!(tree).value[:value]
for i in 1:5000
    tree.value[:d] .= ones(length(eval_tree!(tree).value[:value])) - eval_tree!(tree).value[:value]
    bp_tree!(tree)
end
## maximum(abs(ones(length(eval_tree!(tree).value[:value])) - eval_tree!(tree).value[:value]))

In [9]:
function toTree(op)
    Tree(op, Dict(), [])
end

function toTree(skeleton :: Array)
    op = skeleton[1]
    subs = skeleton[2:end]
    Tree(op, Dict(), [toTree(s) for s in subs])
end

function add!(dict :: Dict, dict1 :: Dict)
    for key in keys(dict1)
        dict[key] = vcat(get!(dict, key, []), dict1[key])
    end
    dict
end

function _index(tree :: Tree, inds :: Array)
    ind = Dict{Any, Any}(tree.op => [tree])
    for ind1 in inds
        add!(ind, ind1)
    end
    ind
end

index(tree :: Tree) = bottom_up(_index, tree)

type Axiom
    tree1 :: Tree
    tree2 :: Tree
    index :: Dict
end

function Base.show(io :: IO, m :: Axiom)
    print(io, "Axiom[")
    print(io, m.tree1)
    print(io, ", ")
    print(io, m.tree2)
    print(io, "]")
end

function beautify(m :: Axiom)
    string("Axiom:\n", beautify(m.tree1, 1), "\n", beautify(m.tree2, 1), "\n")
end

Axiom(tree1 :: Tree, tree2 :: Tree) = Axiom(tree1, tree2, add!(index(tree1), index(tree2)))

Axiom(skeleton1, skeleton2) = Axiom(toTree(skeleton1), toTree(skeleton2))

function push!(index :: Dict, ops :: Array)
    n = length(ops)
    for i in 1:n
        for t in index[i]
            t.op = ops[i]
        end
    end
end

function push!(index :: Dict, ops :: Dict)
    for key in keys(ops)
        if haskey(index, key)
            ts = index[key]
            for t in ts
                t.op = ops[key]
            end
        end
    end
end

function push!(axiom :: Axiom, ops)
    push!(axiom.index, variables)
end

function train!(axiom :: Axiom, variables, n = 1)
    push!(axiom.index, variables)
    init_tree!(axiom.tree1)
    init_tree!(axiom.tree2)
    d1 :: Array{Float64, 1} = axiom.tree1.value[:d]
    d2 :: Array{Float64, 1} = axiom.tree2.value[:d]
    v1 :: Array{Float64, 1} = axiom.tree1.value[:value]
    v2 :: Array{Float64, 1} = axiom.tree2.value[:value]
    for i in 1:n
        eval_tree!(axiom.tree2)
        eval_tree!(axiom.tree1)
        for j in 1:length(d1)
                d1[j] = v2[j] - v1[j]
                d2[j] = -d1[j]
        end
        bp_tree!(axiom.tree1)
        bp_tree!(axiom.tree2)
    end
end

train! (generic function with 2 methods)

In [10]:
Sensor = Class("Sensor", 100)
sensor = Object("sensor", Sensor)
Action = Class("Action", 120)
action = Object("action", Action)
act = DFunction("act", [Sensor], Action)
invact = DFunction("invact", [Action], Sensor)
axiom = Axiom([1, [2, 3]], 3)

Axiom[[1, [2, 3]], 3]

In [11]:
## @time train!(axiom, [invact, act, sensor], 10000)
train!(axiom, [invact, act, sensor], 10000)
## @time foreach(i -> eval_tree!(axiom.tree1), 1:10000)
maximum(abs(eval_tree!(axiom.tree2).value[:value] - eval_tree!(axiom.tree1).value[:value]))
Profile.clear()
@profile train!(axiom, [invact, act, sensor], 1000)
## Profile.print()
## Profile.print(format = :flat)

In [12]:
## Lambda Calculus

Variable = Class("Variable", 100)
LambdaTerm = Class("LambdaTerm", 200)
var2term = DFunction("var2term", [Variable], LambdaTerm)
app = DFunction("app", [LambdaTerm, LambdaTerm], LambdaTerm)
lambda = DFunction("lambda", [Variable, LambdaTerm], LambdaTerm)

## substitution
## t1 [x = t2]
subs = DFunction("subs", [LambdaTerm, Variable, LambdaTerm], LambdaTerm)
## x[x := N]       ≡ N
axiom_s1 = Axiom([subs, [var2term, :v], :v, :t], :t)
## y[x := N]       ≡ y, if x ≠ y
axiom_s2 = Axiom([subs, [var2term, :v1], :v2, :t], [var2term, :v1])
## (M1 M2)[x := N] ≡ (M1[x := N]) (M2[x := N])
axiom_s3 = Axiom([subs, [app, :t1, :t2], :v, :t], [app, [subs, :t1, :v, :t], [subs, :t2, :v, :t]])
## (λx.M)[x := N]  ≡ λx.M
axiom_s4 = Axiom([subs, [lambda, :v, :t1], :v, :t2], [lambda, :v, :t1])
## (λy.M)[x := N]  ≡ λy.(M[x := N]), if x ≠ y, provided y ∉ FV(N)
axiom_s5 = Axiom([subs, [lambda, :v1, :t1], :v2, :t2], [lambda, :v1, [subs, :t1, :v2, :t2]])

## beta-reduction of  ((λV.E) E′)  is E[V := E′].
axiom_beta = Axiom([app, [lambda, :v, :t], :t1], [subs, :t, :v, :t1])
## eta-reduction
axiom_eta = Axiom([lambda, :v, [app, :t, [var2term, :v]]], :t)

Axiom[[Func(lambda){Variable,LambdaTerm->LambdaTerm}, v, [Func(app){LambdaTerm,LambdaTerm->LambdaTerm}, t, [Func(var2term){Variable->LambdaTerm}, v]]], t]

In [13]:
dict = Dict()
dict[:v] = Object("v", Variable)
dict[:v1] = Object("v1", Variable)
dict[:v2] = Object("v2", Variable)
dict[:t] = Object("t", LambdaTerm)
dict[:t1] = Object("t1", LambdaTerm)
dict[:t2] = Object("t2", LambdaTerm)

Obj(t2){LambdaTerm}

In [14]:
## @time train!(axiom_s5, dict, 1000)
train!(axiom_eta, dict, 1000)

In [15]:
type Pair
    class1 :: Class
    class2 :: Class
    pclass :: Class
    
    pair :: DFunction
    first :: DFunction
    second :: DFunction
    first! :: DFunction
    second! :: DFunction
    
    axiom_first :: Axiom
    axiom_second :: Axiom
    axiom_first! :: Axiom
    axiom_second! :: Axiom
end

function Base.show(io :: IO, m :: Pair)
    print(io, string("Module.Pair(", m.class1.class_name, ",", m.class2.class_name, "->", m.pclass.class_name, ")"))
end

function Pair(class1 :: Class, class2 :: Class, pclass :: Class)
    pair = DFunction("pair", [class1, class2], pclass)
    first = DFunction("first", [pclass], class1)
    second = DFunction("second", [pclass], class2)
    first! = DFunction("first!", [pclass, class1], pclass)
    second! = DFunction("second!", [pclass, class2], pclass)
    
    axiom_first = Axiom([first, [pair, :x, :y]], :x)
    axiom_second = Axiom([second, [pair, :x, :y]], :y)
    axiom_first! = Axiom([first!, :xy, :x], [pair, :x, [second, :xy]])
    axiom_second! = Axiom([second!, :xy, :y], [pair, [first, :xy], :y])
    
    Pair(class1, class2, pclass, pair, first, second, first!, second!, axiom_first, axiom_second, axiom_first!, axiom_second!)
end

function Pair(class1 :: Class, class2 :: Class, type_len :: Int64)
    post_name = string("_", class1.class_name, "_", class2.class_name)
    pair_name = string("Pair", post_name)
    pclass = Class(pair_name, type_len)
    Pair(class1, class2, pclass)
end

## @time p = Pair(Sensor, Action, 70)

Pair

In [16]:
type List
    class :: Class
    lclass :: Class
    
    empty :: Object

    cons :: DFunction
    first :: DFunction
    rest :: DFunction
    first! :: DFunction
    rest! :: DFunction
    
    axiom_first :: Axiom
    axiom_rest :: Axiom
    axiom_first! :: Axiom
    axiom_rest! :: Axiom
end

function Base.show(io :: IO, m :: List)
    print(io, string("Module.List(", m.class.class_name, "->", m.lclass.class_name, ")"))
end

function List(class :: Class, lclass :: Class)
    empty = Object("empty", lclass)
    empty.value .= 0.
    
    cons = DFunction("cons", [class, lclass], lclass)
    first = DFunction("first", [lclass], class)
    rest = DFunction("rest", [lclass], lclass)
    first! = DFunction("first!", [lclass, class], lclass)
    rest! = DFunction("rest!", [lclass, lclass], lclass)
    
    axiom_first = Axiom([first, [cons, :x, :xs]], :x)
    axiom_rest = Axiom([rest, [cons, :x, :xs]], :xs)
    axiom_first! = Axiom([first!, :xxs, :x], [cons, :x, [rest, :xxs]])
    axiom_rest! = Axiom([rest!, :xxs, :xs], [cons, [first, :xxs], :xs])
    
    List(class, lclass, empty, cons, first, rest, first!, rest!, axiom_first, axiom_rest, axiom_first!, axiom_rest!)
end

function List(class :: Class, type_len :: Int64)
    list_name = string("List", "_", class.class_name)
    lclass = Class(list_name, type_len)
    List(class, lclass)
end

## @time l = List(Sensor, 70)

List

In [18]:
## Binding, Frame and Environment

Variable = Class("Variable", 100)
Prog = Class("Prog", 100)
Bindingc = Class("Binding", 100)
Binding = Pair(Variable, Prog, Bindingc)
Framec = Class("Frame", 100)
Frame = List(Bindingc, Framec)
Envc = Class("Env", 200)
Env = List(Framec, Envc)

lookup = DFunction("lookup", [Envc, Variable], Prog)
axiom_lookup1 = Axiom([lookup, [Env.cons, [Frame.cons, [Binding.pair, :v1, :t1], :f2], :e2], :v1], :t1)
axiom_lookup2 = Axiom([lookup, [Env.cons, [Frame.cons, [Binding.pair, :v1, :t1], :f2], :e2], :v2], 
                      [lookup, [Env.cons, :f2, :e2], :v2])
axiom_lookup3 = Axiom([lookup, [Env.cons, Frame.empty, :e2], :v1], [lookup, :e2, :v1])

set = DFunction("set", [Envc, Variable, Prog], Envc)
axiom_set1 = Axiom([set, [Env.cons, [Frame.cons, [Binding.pair, :v1, :t1], :f2], :e2], :v1, :t2], 
                   [Env.cons, [Frame.cons, [Binding.pair, :v1, :t2], :f2], :e2])
axiom_set2 = Axiom([set, [Env.cons, [Frame.cons, [Binding.pair, :v1, :t1], :f2], :e2], :v2, :t2], 
                   [Env.cons, [Frame.cons, [Binding.pair, :v1, :t1], 
                                           [Env.first, [set, [Env.cons, :f2, :e2], :v2, :t2]]], 
                              [Env.rest, [set, [Env.cons, :f2, :e2], :v2, :t2]]])
axiom_set3 = Axiom([set, [Env.cons, Frame.empty, :e], :v, :p], [Env.cons, Frame.empty, [set, :e, :v, :p]])

add_binding = DFunction("add_binding", [Framec, Variable, Prog], Framec)
axiom_add_binding = Axiom([add_binding, :f, :v, :p], [Frame.cons, [Binding.pair, :v, :p], :f])
def = DFunction("def", [Envc, Variable, Prog], Envc)
axiom_def = Axiom([def, :e, :v, :p], [Env.first!, :e, [add_binding, [Env.first, :e], :v, :p]])
extend = DFunction("extend", [Envc, Variable, Prog], Envc)
axiom_extend = Axiom([extend, :e, :v, :p], [def, [Env.cons, Env.empty, :e], :v, :p])

var = DFunction("var", [Variable], Prog)
definition = DFunction("definition", [Variable, Prog], Prog) 
assignment = DFunction("assignment", [Variable, Prog], Prog) 
procedure = DFunction("procedure", [Variable, Prog], Prog)
func_call = DFunction("func_call", [Prog, Prog], Prog)
Seq = List(Prog, Prog)

f_eval = DFunction("f_eval", [Prog, Envc], Prog)
s_eval = DFunction("eval", [Prog, Envc], Envc)

axiom_var_s = Axiom([s_eval, [var, :v], :e], :e)
axiom_var_f = Axiom([f_eval, [var, :v], :e], [lookup, :e, :v])
axiom_definition_s = Axiom([s_eval, [definition, :v, :p], :e], [def, [s_eval, :p, :e], :v, [f_eval, :p, :e]])
axiom_definition_f = Axiom([f_eval, [definition, :v, :p], :e], [f_eval, :p, :e])
axiom_assignment_s = Axiom([s_eval, [assignment, :v, :p], :e], [set, [s_eval, :p, :e], :v, [f_eval, :p, :e]])
axiom_assignment_f = Axiom([f_eval, [assignment, :v, :p], :e], [f_eval, :p, :e])
axiom_proc_s = Axiom([s_eval, [procedure, :v, :p], :e], :e)
axiom_proc_f = Axiom([f_eval, [procedure, :v, :p], :e], [procedure, :v, :p])
axiom_func_s1 = Axiom([s_eval, [func_call, :p1, :p2], :e], [s_eval, [func_call, [f_eval, :p1, :e], [f_eval, :p2, :e]], :e])
axiom_func_f1 = Axiom([f_eval, [func_call, :p1, :p2], :e], [f_eval, [func_call, [f_eval, :p1, :e], [f_eval, :p2, :e]], :e])
axiom_func_s2 = Axiom([s_eval, [func_call, [procedure, :v, :p1], :p2], :e], [Env.rest, [s_eval, :p1, [extend, :e, :v, :p2]]])
axiom_func_f2 = Axiom([f_eval, [func_call, [procedure, :v, :p1], :p2], :e], [f_eval, :p1, [extend, :e, :v, :p2]])
axiom_seq_s = Axiom([s_eval, [Seq.cons, :p1, :p2], :e], [s_eval, :p2, [s_eval, :p1, :e]])
axiom_seq_f = Axiom([f_eval, [Seq.cons, :p1, :p2], :e], [f_eval, :p2, [s_eval, :p1, :e]])

Axiom[[Func(f_eval){Prog,Env->Prog}, [Func(cons){Prog,Prog->Prog}, p1, p2], e], [Func(f_eval){Prog,Env->Prog}, p2, [Func(eval){Prog,Env->Env}, p1, e]]]